# Обработка данных:  сортировки, фильтрация и группировка в Pandas

## 1. Статистика в DataFrame

Для дальнейшего изучения будем использовать датасет "Абитуриент":

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv(r"abiturient_fix.csv",
                   encoding="cp1251",
                   sep=";",
                  decimal=",")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12352 entries, 0 to 12351
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   №                        12352 non-null  int64  
 1   ЛД                       12352 non-null  object 
 2   ФИО                      12352 non-null  object 
 3   Статус                   12352 non-null  object 
 4   Приориетет               12352 non-null  object 
 5   Рейтинг                  12352 non-null  float64
 6   Факультет                12352 non-null  object 
 7   Форма обучения           12352 non-null  object 
 8   Период обучения          12352 non-null  object 
 9   Срок обучения            12352 non-null  object 
 10  Курс                     12352 non-null  object 
 11  Код Специальности        12352 non-null  object 
 12  Специальность            12352 non-null  object 
 13  Образовательный уровень  12352 non-null  object 
 14  Комиссия              

В датасете **НЕ ИСПОЛЬЗУЮТСЯ** реальные персональные данные, они рандомизированы.

Дадим краткую харакеристику датасету:

|№|Имя столбца|Содержание столбца|
|:-|:-|:-|
|0|№|Уникальный номер записи в базе данных. Можно использовать для идентификации записи вместо индекса. Это номер заявления|
|1|ЛД|Номер личного дела. Уникальный идентификатор абитуриента. У каждого абитуриента одно личное дело, но в нем может быть несколько заявлений|
|2|ФИО|Фамилия, имя и отчество абитуриента. **Рандомизированы**|
|3|Статус|Статус завяления. Статус "В приказ" означает, что абитуриент зачислен по данному заявлению. В личном деле может быть лишь одно заявление с этим статусом|
|4|Приоритет|Используется при зачислении по нескольким заявлениям. Высший приоритет - первый|
|5|Рейтинг|Вычисляемый системой при подаче заявлений конкурсный балл|
|6|Факультет|Факультет, на котором находится специальность, на которую подано заявление|
|7|Форма обучения|Одна из трёх форм обучения. На разных специальностях могут быть разные формы обучения|
|8|Период обучения|Один из трёх периодов обученияl|   |ect
|9|Срок обучения|Время обучения на избранной специальности|
|11|Курс|Курс зачисления. Обычно это первый курс, но в случае перевода или восстановления студента может быть любой, кроме первого|
|11|Код Специальности|Шифровой код специальности|
|12|Специальность|Наименование специальности с профилем обучения|
|13|Образовательный уровень|Один из трех образовательных уровней|
|14|Комиссия|Подкомиссия приёмной комиссии, осуществляющая приём на данный образовательный уровень или переводы и восстановление студентовo|
|15|Тип док. об обр.|Документ о ранее полученном образовании, на базе которого осуществляется поступление|
|16|Ср. балл|Средний балл в документе о предыдущем образовании|
|17|Награды|Наличие особых заслуг при равенстве баллов3|
|18|Оригинал|Наличие оригинала документа об образовании. При его отсутствии абитуриент не допускается к конкурсу|
|19|Коментарии|Технические замечания сотрудника ПК при отклонении завяления (опционально)|
|20|Бюджет|Наличие права на зачисление на бюджетные места|
|21|Общий конкурс|Участие в конкурсе на общих основаниях|
|22|Целевой конкурс|Участие в целевом конкурсе|
|23|Дата создания|Дата создания заявления|

Конкретные значения можно просмотреть через индек столбца:

In [ ]:
data['ЛД']

0          825ПВ
1          826ПВ
2          827ПВ
3          828ПВ
4          829ПВ
          ...   
12347    14050БС
12348    14051БС
12349    14052БС
12350      4620У
12351    14053БС
Name: ЛД, Length: 12352, dtype: object

Но в случае больших датасетов это представление не даёт полного представления о данных. Для этого можно воспользоваться свойством `.values`:

In [ ]:
data['ЛД'].values

array(['825ПВ', '826ПВ', '827ПВ', ..., '14052БС', '4620У', '14053БС'],
      dtype=object)

а для повторяющихся значений - методами `.unique()` и `.value_counts()`:

In [ ]:
data['Факультет'].unique()

array(['Юрфак', 'Физтех', 'Истфак', 'ФМИТ', 'Экономфак', 'Учфин',
       'Филфак', 'Биофак', 'Химфак', 'УНИЭК', 'ИФКС', 'Пединститут',
       'ФИЯ', 'ФДиПО', 'ГУКЦ', 'ЕУКЦ'], dtype=object)

In [ ]:
data['Факультет'].value_counts()

Факультет
Пединститут    2229
Экономфак      1762
Филфак         1557
ИФКС           1357
Юрфак          1114
Учфин           763
ФИЯ             722
Физтех          698
Истфак          666
ФМИТ            626
Биофак          390
ФДиПО           170
Химфак          133
УНИЭК           130
ГУКЦ             20
ЕУКЦ             15
Name: count, dtype: int64

Первичная обработка данных предполагает применение упорядочения по отдельным показателям, фильтрации по значениям, группировки по критериям.

Некоторые стандартные статистические характеристики датасетов предоставляются `Pandas` встроенными функциями:

|Метод|Назначение|
|:-:|:-|
|`.sum()`|Подсчитывает сумму элементов|
|`.count()`|Подсчитывает количество элементов|
|`.min()`|Находит минимальный элемент|
|`.max()`|Находит максимальный элемент|
|`.mean(skipna=True`)|возвращает среднюю величину для каждого столбца, пропуск `NaN` (по умолчанию `True`)
|`.median(skipna=True)`|возвращает медиану для каждого столбца, пропуск `NaN` (по умолчанию `True`)
|`.mode(dropna=True)`|возвращает медиану для каждого столбца, пропуск `NaN`
|`.var()`|возвращает дисперсию
|`.std()`|возвращает стандартное отклонение
|`.diff()`|возвращает абсолютную разность индексируемых значений
|`.unique()`|возвращает список уникальных значений
|`.value_counts()`|подсчитывает количество каждого уникального значения
|`.describe()`|дает расширенное описание


In [3]:
data['Рейтинг'].describe()

,Рейтинг
count,12352.000000
mean,81.492247
std,15.647283
min,0.870000
25%,74.560000
50%,83.900000
75%,91.420000
max,136.260000


In [ ]:
data['Рейтинг'].sum()

1006592.24

Иногда в процессе обработки требуется отобрать данные по какому либо критерию, иными словами, осуществить фильтрацию данных.

Отберем данные о зачисленных абитуриентах на основании статуса заявления:

In [ ]:
data['Статус'].unique()

array(['В приказ', 'Отказ', 'Отклонено', 'Отменено', 'Отчислен',
       'Задержано', 'Аннулировано', 'Допущено', 'В приказ по квоте'],
      dtype=object)

Абитуриенты, которые были зачислены, имеют статус заявления "В приказ":

In [ ]:
data['Статус'] == "В приказ"

0        True
1        True
2        True
3        True
4        True
         ... 
12347    True
12348    True
12349    True
12350    True
12351    True
Name: Статус, Length: 12352, dtype: bool

Это бинарный массив. Если его использовать для индексации датасета (так называемое логическое индексирование), то получим только те строки, для которых в бинарном массиве будет `True`:

In [ ]:
data[data['Статус'] == "В приказ"]

,№,ЛД,ФИО,Статус,Приориетет,Рейтинг,Факультет,Форма обучения,Период обучения,Срок обучения,...,Комиссия,Тип док. об обр.,Ср. балл,Награды,Оригинал,Коментарии,Бюджет,Общий конкурс,Целевой конкурс,Дата создания
0,28863,825ПВ,Иннокентиевская Виктория Дмитриeвна,В приказ,Первый,84.54,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,95.08,без отличия,нет,NaN,нет,да,нет,09.01.2019 9:33
1,28864,826ПВ,Серафимова Ирина Ивановна,В приказ,Первый,73.00,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,72.00,без отличия,да,NaN,нет,да,нет,09.01.2019 15:15
2,28865,827ПВ,Кондратенко Зинаида Трофимовна,В приказ,Первый,75.00,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,4.00,без отличия,нет,NaN,нет,да,нет,10.01.2019 14:41
3,28866,828ПВ,Андреевский Андрей Игнатович,В приказ,Первый,72.30,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,71.60,без отличия,да,NaN,нет,да,нет,10.01.2019 15:08
4,28867,829ПВ,Кондратовский Петр Тимофеeвич,В приказ,Первый,85.00,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,5.00,без отличия,да,NaN,нет,да,нет,10.01.2019 15:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12347,41392,14050БС,Терентиевич Ирина Ивановна,В приказ,Первый,66.88,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,Бакалавры/Специалисты,Диплом квалифицированного работника,4.57,без отличия,да,NaN,да,нет,да,08.10.2019 14:51
12348,41393,14051БС,Петровский Алексей Кондратович,В приказ,Первый,70.28,Пединститут,заочная,Нормативный,5 лет 00 мес.,...,Бакалавры/Специалисты,Аттестат о среднем общем образовании,4.42,без отличия,да,NaN,нет,да,нет,08.10.2019 14:56
12349,41394,14052БС,Семенова Инна Алексеeвна,В приказ,Первый,96.02,ФМИТ,заочная,Нормативный,5 лет 00 мес.,...,Бакалавры/Специалисты,Аттестат о среднем общем образовании,4.88,без отличия,да,NaN,да,да,нет,08.10.2019 15:09
12350,41397,4620У,Петренко Лидия Петровна,В приказ,Первый,71.80,Экономфак,заочная,Ускоренный,2 года 6 мес.,...,СПО,Диплом о среднем профессиональном образовании,3.35,без отличия,да,NaN,да,да,нет,08.10.2019 15:44


Аналогичным образом можно не только создавать новые датасеты, но очищать старые, используя `.drop()`.

Удалим все данные о переводах и восстановлениях из датасета.

Бинарный массив:

In [ ]:
data['Комиссия'] == 'Переводы, восстановления'

0         True
1         True
2         True
3         True
4         True
         ...  
12347    False
12348    False
12349    False
12350    False
12351    False
Name: Комиссия, Length: 12352, dtype: bool

Соответствующие строки датасета:

In [ ]:
data[data['Комиссия'] == 'Переводы, восстановления']

,№,ЛД,ФИО,Статус,Приориетет,Рейтинг,Факультет,Форма обучения,Период обучения,Срок обучения,...,Комиссия,Тип док. об обр.,Ср. балл,Награды,Оригинал,Коментарии,Бюджет,Общий конкурс,Целевой конкурс,Дата создания
0,28863,825ПВ,Иннокентиевская Виктория Дмитриeвна,В приказ,Первый,84.54,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,95.08,без отличия,нет,NaN,нет,да,нет,09.01.2019 9:33
1,28864,826ПВ,Серафимова Ирина Ивановна,В приказ,Первый,73.00,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,72.00,без отличия,да,NaN,нет,да,нет,09.01.2019 15:15
2,28865,827ПВ,Кондратенко Зинаида Трофимовна,В приказ,Первый,75.00,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,4.00,без отличия,нет,NaN,нет,да,нет,10.01.2019 14:41
3,28866,828ПВ,Андреевский Андрей Игнатович,В приказ,Первый,72.30,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,71.60,без отличия,да,NaN,нет,да,нет,10.01.2019 15:08
4,28867,829ПВ,Кондратовский Петр Тимофеeвич,В приказ,Первый,85.00,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,5.00,без отличия,да,NaN,нет,да,нет,10.01.2019 15:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11886,40731,3378ПВ,Кириллов Петр Кириллович,В приказ,Первый,88.00,Филфак,заочная,Нормативный,4 года 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,5.00,без отличия,нет,NaN,нет,да,нет,24.08.2019 9:22
11887,40733,3379ПВ,Владимирович Фаина Владимировна,В приказ,Первый,74.52,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,4.44,без отличия,да,NaN,нет,да,нет,24.08.2019 10:12
11888,40734,3380ПВ,Ивановская Ольга Глебовна,В приказ,Первый,76.32,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,4.29,без отличия,да,NaN,нет,да,нет,24.08.2019 10:26
11889,40735,3381ПВ,Петрович Галина Семеновна,В приказ,Первый,86.90,Филфак,заочная,Нормативный,4 года 00 мес.,...,"Переводы, восстановления",Аттестат о среднем общем образовании,4.69,без отличия,да,NaN,нет,да,нет,24.08.2019 10:39


Их индексы:

In [ ]:
data[data['Комиссия'] == 'Переводы, восстановления'].index

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       11881, 11882, 11883, 11884, 11885, 11886, 11887, 11888, 11889, 11890],
      dtype='int64', length=2588)

"Очищенный" датасет (удаляем строки):

In [ ]:
data = data.drop(data[data['Комиссия'] == 'Переводы, восстановления'].index, axis=0)
data.reset_index(drop=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9764 entries, 2353 to 12351
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   №                        9764 non-null   int64  
 1   ЛД                       9764 non-null   object 
 2   ФИО                      9764 non-null   object 
 3   Статус                   9764 non-null   object 
 4   Приориетет               9764 non-null   object 
 5   Рейтинг                  9764 non-null   float64
 6   Факультет                9764 non-null   object 
 7   Форма обучения           9764 non-null   object 
 8   Период обучения          9764 non-null   object 
 9   Срок обучения            9764 non-null   object 
 10  Курс                     9764 non-null   object 
 11  Код Специальности        9764 non-null   object 
 12  Специальность            9764 non-null   object 
 13  Образовательный уровень  9764 non-null   object 
 14  Комиссия                 

Метод `.reset_index()` применяется для сброса индекса. При удалении строк его применение позволяет создать новый непрерывный индекс из целых чисел.

Кроме того, датасеты могут содержать пропущенные данные. Они интерпретируются как подтип типа `float`. Это позволяет использовать обобщающий тип `float` для столбцов с пропущенными данными. Некоторые функции и иметоды позволяют учитывать наличие пропущенных данных и выполняют соответствующие вычисления. Однако может потребоваться обработка пропущенных значений. Есть несколько возможных сценариев:

- интерпретация пропущенных значений и заполнение значениями по умолчанию (в некоторых случаях, например, 0);
- интерполяция пропущенных значений, например с использованием соседних значений, сплайн-функций и т.д.;
- фильтрация пропущенных значений, что, безусловно, приведет к сокращению датасета.

В нашем датасете имеются пропущенные значения в столбце с коментариями. При необходимости их можно заполнить пустыми строками:

In [ ]:
df_ = data['Коментарии']

df_.info()
print()
df_ = df_.fillna("")
df_.info()

<class 'pandas.core.series.Series'>
Index: 9764 entries, 2353 to 12351
Series name: Коментарии
Non-Null Count  Dtype 
--------------  ----- 
4481 non-null   object
dtypes: object(1)
memory usage: 152.6+ KB

<class 'pandas.core.series.Series'>
Index: 9764 entries, 2353 to 12351
Series name: Коментарии
Non-Null Count  Dtype 
--------------  ----- 
9764 non-null   object
dtypes: object(1)
memory usage: 152.6+ KB


В случае если у нас данные для которых соседние значения в столбце где нужно заполнить пропуски близки, то можно использовать параметр method в методе `fillna()`.

Синтаксис:
`dataframe.fillna(value, method, axis, inplace, limit, downcast)`

В качестве аргументов параметра

|Параметр|Допустимые значения/тип передаваемого объекта|Назначение|
|:-:|:-|:-|
|`value`| Number, String, Dictionary, Series, DataFrame| Задает значения для замены пропусков. Можно как все пропуски заменять одним значением, так и использовать для каждой строки/столбца заданное значение.|
`method`| 'backfill', 'bfill', 'pad', 'ffill', None| Опционально. Определяет каким методом заполнять пропуски. |
`axis`| 0, 1, 'index', 'columns'| Опционально, значение по умолчанию равно 0. Ось вдоль которой заполняются пропуски.|
`inplace`| True, False| Опционально, значение по умолчанию равно False. Если True: замена выполняется в текущем фрейме данных. Если False: возвращает копию, в которой выполнена замена.|
`limit`| Number, None| Опционально, значение по умолчанию равно None. Указывает максимальное количество нулевых значений для заполнения (если указан метод).|
`downcast`| Number, None| Опционально, словарь значений для заполнения для определенных типов данных.|


Кроме того можно очистить датасет от данных, содержащих пропущенные значения. Удаление будет происходить для строки/столбца состоящего исключительно из пропущенных значений(именованный параметр `how='all'`), либо содержащих хотя бы одно пропущенное значение (`how='any'`).

Удалим все строки, не содержащие коментарии:

In [ ]:
df_ = data.dropna(axis=0, how='any')

df_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4481 entries, 2355 to 12342
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   №                        4481 non-null   int64  
 1   ЛД                       4481 non-null   object 
 2   ФИО                      4481 non-null   object 
 3   Статус                   4481 non-null   object 
 4   Приориетет               4481 non-null   object 
 5   Рейтинг                  4481 non-null   float64
 6   Факультет                4481 non-null   object 
 7   Форма обучения           4481 non-null   object 
 8   Период обучения          4481 non-null   object 
 9   Срок обучения            4481 non-null   object 
 10  Курс                     4481 non-null   object 
 11  Код Специальности        4481 non-null   object 
 12  Специальность            4481 non-null   object 
 13  Образовательный уровень  4481 non-null   object 
 14  Комиссия                 

Для фильтрации пропущенных значений можно применять методы `.isna()` и `.notna()`:

In [ ]:
data[data['Коментарии'].isna()][['ЛД', 'ФИО', 'Статус', 'Коментарии']]

,ЛД,ФИО,Статус,Коментарии
2353,10121БС,Дмитриевич Иван Андреeвич,Аннулировано,NaN
2354,10121БС,Дмитриевич Иван Андреeвич,В приказ,NaN
2356,10122БС,Семенович Степан Степанович,В приказ,NaN
2357,4046У,Иванов Кондрат Игнатович,В приказ,NaN
2358,10123БС,Терентиев Борис Алексеeвич,В приказ,NaN
...,...,...,...,...
12347,14050БС,Терентиевич Ирина Ивановна,В приказ,NaN
12348,14051БС,Петровский Алексей Кондратович,В приказ,NaN
12349,14052БС,Семенова Инна Алексеeвна,В приказ,NaN
12350,4620У,Петренко Лидия Петровна,В приказ,NaN


In [ ]:
data[data['Коментарии'].notna()][['ЛД', 'ФИО', 'Статус', 'Коментарии']]

,ЛД,ФИО,Статус,Коментарии
2355,10122БС,Семенович Степан Степанович,Отклонено,Отклонено в связи с рекомендацией по старшему ...
2359,10123БС,Терентиев Борис Алексеeвич,Отклонено,Отклонено в связи с рекомендацией по старшему ...
2361,10125БС,Александровский Иван Александрович,Отменено,Не предоставил полный комплект документов
2366,10127БС,Степанов Тимофей Борисович,Отклонено,Отклонено в связи с рекомендацией по старшему ...
2368,10128БС,Кондратенко Дмитрий Кириллович,Отклонено,Отказался от участия в конкурсе
...,...,...,...,...
12317,14023БС,Трофимович Марина Кирилловна,Отменено,Ошибка оператора
12320,14026БС,Тимофеенко Кондрат Дмитриeвич,Отменено,техническая ошибка
12327,14033БС,Степаненко Мария Кондратовна,Отклонено,нет договора
12334,14038БС,Петров Трофим Николаeвич,Отказ,не набран проходной балл


К объектам DataFrame также можно применять универсальные функции NumPy.

In [ ]:
np.mean(data['Рейтинг'])

82.5446825071692

Иногда это может привести к возникновению пропущенных значений

In [ ]:
np.log(data['Рейтинг'] - 75)

C:\Users\LOKI\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\LOKI\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


2353     3.090133
2354     3.090133
2355     2.961141
2356     2.961141
2357     2.736962
           ...   
12347         NaN
12348         NaN
12349    3.045474
12350         NaN
12351    1.449269
Name: Рейтинг, Length: 9764, dtype: float64

В модуле `Pandas` также реализованы и собственные методы для отдельных видов данных, в частности для даты/времени и строк. Рассмотрим данные столбца `Дата создания` нашего датасета:

In [ ]:
data['Дата создания']

2353      24.06.2019 9:31
2354      24.06.2019 9:34
2355      24.06.2019 9:39
2356      24.06.2019 9:32
2357      24.06.2019 9:59
               ...       
12347    08.10.2019 14:51
12348    08.10.2019 14:56
12349    08.10.2019 15:09
12350    08.10.2019 15:44
12351    08.10.2019 17:04
Name: Дата создания, Length: 9764, dtype: object

In [ ]:
print(data['Дата создания'].dtype)

object


Использование строковой величины в качестве даты приведет к ошибкам сравнения. Разделим этот столбец на два: для даты и времени:

In [ ]:
data[['Дата', 'Время']] = data['Дата создания'].str.split(expand=True)

data

,№,ЛД,ФИО,Статус,Приориетет,Рейтинг,Факультет,Форма обучения,Период обучения,Срок обучения,...,Ср. балл,Награды,Оригинал,Коментарии,Бюджет,Общий конкурс,Целевой конкурс,Дата создания,Дата,Время
2353,31288,10121БС,Дмитриевич Иван Андреeвич,Аннулировано,Первый,96.98,ФИЯ,очная,Нормативный,4 года 00 мес.,...,4.82,без отличия,да,NaN,да,да,нет,24.06.2019 9:31,24.06.2019,9:31
2354,31290,10121БС,Дмитриевич Иван Андреeвич,В приказ,Второй,96.98,ФИЯ,очная,Нормативный,4 года 00 мес.,...,4.82,без отличия,да,NaN,да,да,нет,24.06.2019 9:34,24.06.2019,9:34
2355,31291,10122БС,Семенович Степан Степанович,Отклонено,Второй,94.32,Филфак,очная,Нормативный,4 года 00 мес.,...,4.38,без отличия,да,Отклонено в связи с рекомендацией по старшему ...,да,да,нет,24.06.2019 9:39,24.06.2019,9:39
2356,31289,10122БС,Семенович Степан Степанович,В приказ,Первый,94.32,Филфак,очная,Нормативный,5 лет 00 мес.,...,4.38,без отличия,да,NaN,да,да,нет,24.06.2019 9:32,24.06.2019,9:32
2357,31292,4046У,Иванов Кондрат Игнатович,В приказ,Первый,90.44,ФМИТ,заочная,Ускоренный,3 года 00 мес.,...,4.48,без отличия,да,NaN,нет,да,нет,24.06.2019 9:59,24.06.2019,9:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12347,41392,14050БС,Терентиевич Ирина Ивановна,В приказ,Первый,66.88,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,4.57,без отличия,да,NaN,да,нет,да,08.10.2019 14:51,08.10.2019,14:51
12348,41393,14051БС,Петровский Алексей Кондратович,В приказ,Первый,70.28,Пединститут,заочная,Нормативный,5 лет 00 мес.,...,4.42,без отличия,да,NaN,нет,да,нет,08.10.2019 14:56,08.10.2019,14:56
12349,41394,14052БС,Семенова Инна Алексеeвна,В приказ,Первый,96.02,ФМИТ,заочная,Нормативный,5 лет 00 мес.,...,4.88,без отличия,да,NaN,да,да,нет,08.10.2019 15:09,08.10.2019,15:09
12350,41397,4620У,Петренко Лидия Петровна,В приказ,Первый,71.80,Экономфак,заочная,Ускоренный,2 года 6 мес.,...,3.35,без отличия,да,NaN,да,да,нет,08.10.2019 15:44,08.10.2019,15:44


Преобразуем столбец `'Дата создания'` в формат `datetime`:

In [ ]:
data['Дата создания'] = pd.to_datetime(data['Дата создания'], dayfirst=True, format='%d.%m.%Y %H:%M')

data

,№,ЛД,ФИО,Статус,Приориетет,Рейтинг,Факультет,Форма обучения,Период обучения,Срок обучения,...,Ср. балл,Награды,Оригинал,Коментарии,Бюджет,Общий конкурс,Целевой конкурс,Дата создания,Дата,Время
2353,31288,10121БС,Дмитриевич Иван Андреeвич,Аннулировано,Первый,96.98,ФИЯ,очная,Нормативный,4 года 00 мес.,...,4.82,без отличия,да,NaN,да,да,нет,2019-06-24 09:31:00,24.06.2019,9:31
2354,31290,10121БС,Дмитриевич Иван Андреeвич,В приказ,Второй,96.98,ФИЯ,очная,Нормативный,4 года 00 мес.,...,4.82,без отличия,да,NaN,да,да,нет,2019-06-24 09:34:00,24.06.2019,9:34
2355,31291,10122БС,Семенович Степан Степанович,Отклонено,Второй,94.32,Филфак,очная,Нормативный,4 года 00 мес.,...,4.38,без отличия,да,Отклонено в связи с рекомендацией по старшему ...,да,да,нет,2019-06-24 09:39:00,24.06.2019,9:39
2356,31289,10122БС,Семенович Степан Степанович,В приказ,Первый,94.32,Филфак,очная,Нормативный,5 лет 00 мес.,...,4.38,без отличия,да,NaN,да,да,нет,2019-06-24 09:32:00,24.06.2019,9:32
2357,31292,4046У,Иванов Кондрат Игнатович,В приказ,Первый,90.44,ФМИТ,заочная,Ускоренный,3 года 00 мес.,...,4.48,без отличия,да,NaN,нет,да,нет,2019-06-24 09:59:00,24.06.2019,9:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12347,41392,14050БС,Терентиевич Ирина Ивановна,В приказ,Первый,66.88,Юрфак,заочная,Нормативный,5 лет 00 мес.,...,4.57,без отличия,да,NaN,да,нет,да,2019-10-08 14:51:00,08.10.2019,14:51
12348,41393,14051БС,Петровский Алексей Кондратович,В приказ,Первый,70.28,Пединститут,заочная,Нормативный,5 лет 00 мес.,...,4.42,без отличия,да,NaN,нет,да,нет,2019-10-08 14:56:00,08.10.2019,14:56
12349,41394,14052БС,Семенова Инна Алексеeвна,В приказ,Первый,96.02,ФМИТ,заочная,Нормативный,5 лет 00 мес.,...,4.88,без отличия,да,NaN,да,да,нет,2019-10-08 15:09:00,08.10.2019,15:09
12350,41397,4620У,Петренко Лидия Петровна,В приказ,Первый,71.80,Экономфак,заочная,Ускоренный,2 года 6 мес.,...,3.35,без отличия,да,NaN,да,да,нет,2019-10-08 15:44:00,08.10.2019,15:44


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9764 entries, 2353 to 12351
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   №                        9764 non-null   int64         
 1   ЛД                       9764 non-null   object        
 2   ФИО                      9764 non-null   object        
 3   Статус                   9764 non-null   object        
 4   Приориетет               9764 non-null   object        
 5   Рейтинг                  9764 non-null   float64       
 6   Факультет                9764 non-null   object        
 7   Форма обучения           9764 non-null   object        
 8   Период обучения          9764 non-null   object        
 9   Срок обучения            9764 non-null   object        
 10  Курс                     9764 non-null   object        
 11  Код Специальности        9764 non-null   object        
 12  Специальность            9764 non-n

## 2. Операции извлечения данных

По умолчанию `Pandas` отображает только чать данных датасета. Для строк отображаются первые пять и последние пять.

Для отображения больших `DataFrame` можно применять настройки:

In [ ]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 30)

In [ ]:
data.head()

,№,ЛД,ФИО,Статус,Приориетет,...,Общий конкурс,Целевой конкурс,Дата создания,Дата,Время
2353,31288,10121БС,Дмитриевич Иван Андреeвич,Аннулировано,Первый,...,да,нет,2019-06-24 09:31:00,24.06.2019,9:31
2354,31290,10121БС,Дмитриевич Иван Андреeвич,В приказ,Второй,...,да,нет,2019-06-24 09:34:00,24.06.2019,9:34
2355,31291,10122БС,Семенович Степан Степанович,Отклонено,Второй,...,да,нет,2019-06-24 09:39:00,24.06.2019,9:39
2356,31289,10122БС,Семенович Степан Степанович,В приказ,Первый,...,да,нет,2019-06-24 09:32:00,24.06.2019,9:32
2357,31292,4046У,Иванов Кондрат Игнатович,В приказ,Первый,...,да,нет,2019-06-24 09:59:00,24.06.2019,9:59


Можно указать целое число (по умолчанию не более 60) в качестве параметра для вывода соответствующего числа строк. Выведем 60 последних строк датасета методом `.tail()`:

In [ ]:
data.tail(60)

,№,ЛД,ФИО,Статус,Приориетет,...,Общий конкурс,Целевой конкурс,Дата создания,Дата,Время
12292,41319,14005БС(ВВ),Степановский Кондрат Кондратович,В приказ,Первый,...,да,нет,2019-09-05 15:28:00,05.09.2019,15:28
12293,41320,4615У,Семенович Александра Кирилловна,В приказ,Первый,...,да,нет,2019-09-05 15:29:00,05.09.2019,15:29
...,...,...,...,...,...,...,...,...,...,...,...
12350,41397,4620У,Петренко Лидия Петровна,В приказ,Первый,...,да,нет,2019-10-08 15:44:00,08.10.2019,15:44
12351,41399,14053БС,Тимофеенко Владимир Трофимович,В приказ,Первый,...,да,нет,2019-10-08 17:04:00,08.10.2019,17:04


Выбрать заданное число строк случайным образом можно с помощью:

In [ ]:
data.sample(5)

In [ ]:
data.sample(5, random_state = 123)

Чаще всего данные извлекаются согласно условиям, накладываемым на значения столбцов:

In [ ]:
data['Рейтинг'] > 100

2353     False
2354     False
         ...  
12350    False
12351    False
Name: Рейтинг, Length: 9764, dtype: bool

In [ ]:
data[data['Рейтинг'] > 100]

,№,ЛД,ФИО,Статус,Приориетет,...,Общий конкурс,Целевой конкурс,Дата создания,Дата,Время
2378,31310,10133БС,Андреева Наталья Владимировна,Отклонено,Первый,...,да,нет,2019-06-24 10:38:00,24.06.2019,10:38
2379,31314,10133БС,Андреева Наталья Владимировна,Отклонено,Второй,...,да,нет,2019-06-24 10:39:00,24.06.2019,10:39
...,...,...,...,...,...,...,...,...,...,...,...
12052,41000,13847БС,Петренко Людмила Владимировна,В приказ,Первый,...,да,нет,2019-09-03 11:42:00,03.09.2019,11:42
12283,41311,4613У,Терентиевич Екатерина Александровна,В приказ,Первый,...,да,нет,2019-09-05 14:11:00,05.09.2019,14:11


Для фильтрации можно использовать сложные условия. Отберем только тех студентов, кто зачислен в бакалавриат:

In [ ]:
data['Комиссия'].unique()

array(['Бакалавры/Специалисты', 'СПО', 'Магистры'], dtype=object)

In [ ]:
df = data[
            (data['Статус'] == "В приказ") &
            (data['Комиссия'] == "Бакалавры/Специалисты")
         ]

df

,№,ЛД,ФИО,Статус,Приориетет,...,Общий конкурс,Целевой конкурс,Дата создания,Дата,Время
2354,31290,10121БС,Дмитриевич Иван Андреeвич,В приказ,Второй,...,да,нет,2019-06-24 09:34:00,24.06.2019,9:34
2356,31289,10122БС,Семенович Степан Степанович,В приказ,Первый,...,да,нет,2019-06-24 09:32:00,24.06.2019,9:32
...,...,...,...,...,...,...,...,...,...,...,...
12349,41394,14052БС,Семенова Инна Алексеeвна,В приказ,Первый,...,да,нет,2019-10-08 15:09:00,08.10.2019,15:09
12351,41399,14053БС,Тимофеенко Владимир Трофимович,В приказ,Первый,...,да,нет,2019-10-08 17:04:00,08.10.2019,17:04


Найдем студентов которые поступали либо на УНИЭК, либо на ФДПО и были зачислены:

In [ ]:
data['Факультет'].unique()

array(['ФИЯ', 'Филфак', 'ФМИТ', 'Учфин', 'Экономфак', 'Юрфак', 'Биофак',
       'Пединститут', 'Химфак', 'Истфак', 'ФДиПО', 'УНИЭК', 'ИФКС',
       'Физтех', 'ГУКЦ', 'ЕУКЦ'], dtype=object)

In [ ]:
data[
        (data['Статус'] == 'В приказ') &
            (
                (data['Факультет'] == 'УНИЭК') |
                 (data['Факультет'] == 'ФДиПО')
            )
    ]

,№,ЛД,ФИО,Статус,Приориетет,...,Общий конкурс,Целевой конкурс,Дата создания,Дата,Время
2415,31347,10151БС,Андреенко Иван Иванович,В приказ,Первый,...,да,нет,2019-06-24 12:08:00,24.06.2019,12:08
2480,31411,2922М,Степанович Александр Степанович,В приказ,Первый,...,да,нет,2019-06-24 15:18:00,24.06.2019,15:18
...,...,...,...,...,...,...,...,...,...,...,...
12243,41259,4343М,Алексеевская Тамара Николаeвна,В приказ,Первый,...,да,нет,2019-09-05 09:49:00,05.09.2019,9:49
12262,41280,13982БС,Семеновская Мария Терентиeвна,В приказ,Первый,...,да,нет,2019-09-05 11:27:00,05.09.2019,11:27


Как альтернативу можно использовать метод `.query()`, который получает строку с запросом:

In [ ]:
data.query('Статус == "В приказ"')

,№,ЛД,ФИО,Статус,Приориетет,...,Общий конкурс,Целевой конкурс,Дата создания,Дата,Время
2354,31290,10121БС,Дмитриевич Иван Андреeвич,В приказ,Второй,...,да,нет,2019-06-24 09:34:00,24.06.2019,9:34
2356,31289,10122БС,Семенович Степан Степанович,В приказ,Первый,...,да,нет,2019-06-24 09:32:00,24.06.2019,9:32
...,...,...,...,...,...,...,...,...,...,...,...
12350,41397,4620У,Петренко Лидия Петровна,В приказ,Первый,...,да,нет,2019-10-08 15:44:00,08.10.2019,15:44
12351,41399,14053БС,Тимофеенко Владимир Трофимович,В приказ,Первый,...,да,нет,2019-10-08 17:04:00,08.10.2019,17:04


In [ ]:
data.query("Статус == 'В приказ' & (Факультет == 'УНИЭК' | 'Факультет' == 'ФДиПО')")

,№,ЛД,ФИО,Статус,Приориетет,...,Общий конкурс,Целевой конкурс,Дата создания,Дата,Время
2720,40520,10304БС,Глебовская Людмила Андреeвна,В приказ,Первый,...,да,нет,2019-07-29 10:54:00,29.07.2019,10:54
3039,40525,10475БС,Глебова Ирина Кирилловна,В приказ,Первый,...,да,нет,2019-07-29 10:54:00,29.07.2019,10:54
...,...,...,...,...,...,...,...,...,...,...,...
11736,40524,13718БС,Кондратович Вера Иннокентиeвна,В приказ,Первый,...,нет,нет,2019-07-29 10:54:00,29.07.2019,10:54
12187,41187,4602У,Степановский Андрей Петрович,В приказ,Первый,...,да,нет,2019-09-04 13:06:00,04.09.2019,13:06


Можно идентифицировать записи, относящиеся к одному объекту. Найдем школьника с самым высоким баллом и просмотрим, куда он собирался поступать.

In [ ]:
df = data[data['Комиссия'] == "Бакалавры/Специалисты"]
n = df[df['Рейтинг'] == df['Рейтинг'].max()]['ЛД'].unique()

n = n[2]

In [ ]:
data[data['ЛД'] == n][['Статус', 'Факультет', 'Специальность']]

,Статус,Факультет,Специальность
2900,В приказ,ФИЯ,Лингвистика (Профиль: Теория и методика препод...
2901,Отклонено,ФИЯ,Филология (Профиль: Зарубежная филология (англ...


## 3. Группировка данных

В библиотеке *Pandas* поддерживаются гибкие возможности группировки данных подобно тому, как это реализовано в *SQL* и сводных таблицах *MS Excel*.

Например, если объект `DataFrame` содержит повторяющиеся данные в столбце, то можно выполнить группировку.

In [ ]:
data.columns

Index(['№', 'ЛД', 'ФИО', 'Статус', 'Приориетет', 'Рейтинг', 'Факультет',
       'Форма обучения', 'Период обучения', 'Срок обучения', 'Курс',
       'Код Специальности', 'Специальность', 'Образовательный уровень',
       'Комиссия', 'Тип док. об обр.', 'Ср. балл', 'Награды', 'Оригинал',
       'Коментарии', 'Бюджет', 'Общий конкурс', 'Целевой конкурс',
       'Дата создания', 'Дата', 'Время'],
      dtype='object')

In [ ]:
df = data[data['Комиссия'] == "Бакалавры/Специалисты"][['№', 'ЛД', 'ФИО', 'Статус', 'Рейтинг','Факультет',
                                                        'Форма обучения', 'Код Специальности',
                                                        'Специальность', 'Дата создания']]

df

,№,ЛД,ФИО,Статус,Рейтинг,Факультет,Форма обучения,Код Специальности,Специальность,Дата создания
2353,31288,10121БС,Дмитриевич Иван Андреeвич,Аннулировано,96.98,ФИЯ,очная,45.03.01,Филология (Профиль: Зарубежная филология (англ...,2019-06-24 09:31:00
2354,31290,10121БС,Дмитриевич Иван Андреeвич,В приказ,96.98,ФИЯ,очная,45.03.02,Лингвистика (Профиль: Теория и методика препод...,2019-06-24 09:34:00
...,...,...,...,...,...,...,...,...,...,...
12349,41394,14052БС,Семенова Инна Алексеeвна,В приказ,96.02,ФМИТ,заочная,44.03.05,Педагогическое образование (Профиль: Математик...,2019-10-08 15:09:00
12351,41399,14053БС,Тимофеенко Владимир Трофимович,В приказ,79.26,Истфак,заочная,41.03.05,Международные отношения,2019-10-08 17:04:00


In [ ]:
groups = df[['ЛД', 'Рейтинг', 'Факультет','Форма обучения', 'Дата создания']].groupby('Факультет')
list(groups)

[('Биофак',
              ЛД  Рейтинг Факультет Форма обучения       Дата создания
  2366   10127БС    93.52    Биофак          очная 2019-06-24 10:14:00
  2378   10133БС   104.70    Биофак          очная 2019-06-24 10:38:00
  ...        ...      ...       ...            ...                 ...
  12320  14026БС    84.16    Биофак        заочная 2019-09-07 11:46:00
  12325  14031БС    86.72    Биофак        заочная 2019-09-07 14:13:00
  
  [243 rows x 5 columns]),
 ('ГУКЦ',
              ЛД  Рейтинг Факультет Форма обучения       Дата создания
  3365   10646БС    64.28      ГУКЦ        заочная 2019-06-27 13:38:00
  3389   10658БС    75.10      ГУКЦ        заочная 2019-06-27 13:58:00
  ...        ...      ...       ...            ...                 ...
  11215  13459БС    76.72      ГУКЦ        заочная 2019-09-03 11:40:00
  12079  13868БС    61.00      ГУКЦ        заочная 2019-09-03 13:40:00
  
  [13 rows x 5 columns]),
 ('ИФКС',
                  ЛД  Рейтинг Факультет Форма обучения   

Группы представляют собой специальный объект `pandas.core.groupby.generic.DataFrameGroupBy`. Для него реализованы некоторые методы:

In [ ]:
groups.size()

Факультет
Биофак        243
ГУКЦ           13
             ... 
Экономфак    1213
Юрфак         705
Length: 15, dtype: int64

Задача, решаемая группировкой - переход к укрупненным данным в группах. Но для этого необходимо корректно указать, как различные данные в строках будут сагреггированы в групповые оценки. В противном случае это вызовет ошибки:

In [ ]:
groups.mean()

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
groups.max()

,ЛД,Рейтинг,Форма обучения,Дата создания
Факультет,,,,
Биофак,14031БС,107.0,очная,2019-09-07 14:13:00
ГУКЦ,13868БС,85.9,заочная,2019-09-03 13:40:00
...,...,...,...,...
Экономфак,14049БС,107.0,очная,2019-10-08 14:51:00
Юрфак,14050БС,107.0,очная,2019-10-08 15:28:00


Данные каждого столбца могут обрабатываться целым набором методов.

In [ ]:
groups.agg(['min', 'max', 'count'])

ЛД                Рейтинг         ... Форма обучения        \
               min      max count     min    max  ...            max count   
Факультет                                         ...                        
Биофак     10127БС  14031БС   243   15.00  107.0  ...          очная   243   
ГУКЦ       10646БС  13868БС    13   15.60   85.9  ...        заочная    13   
...            ...      ...   ...     ...    ...  ...            ...   ...   
Экономфак  10125БС  14049БС  1213   12.68  107.0  ...          очная  1213   
Юрфак      10126БС  14050БС   705    0.87  107.0  ...          очная   705   

                Дата создания                            
                          min                 max count  
Факультет                                                
Биофак    2019-06-24 10:14:00 2019-09-07 14:13:00   243  
ГУКЦ      2019-06-27 13:38:00 2019-09-03 13:40:00    13  
...                       ...                 ...   ...  
Экономфак 2019-06-24 10:08:00 2019-10-08 14:51:00  1213  
Юрфак     2019-06-24 10:12:00 2019-10-08 15:28:00   705  

[15 rows x 12 columns]

Для обеспечения корректной работы и наглядного представления методы можно указывать в виде словаря:

In [ ]:
d = {'ЛД':['count'], 'Рейтинг':['min', 'mean', 'max'], 'Дата создания':['min', 'max']}

groups.agg(d)

ЛД Рейтинг                         Дата создания  \
          count     min       mean    max                 min   
Факультет                                                       
Биофак      243   15.00  85.832099  107.0 2019-06-24 10:14:00   
ГУКЦ         13   15.60  63.852308   85.9 2019-06-27 13:38:00   
...         ...     ...        ...    ...                 ...   
Экономфак  1213   12.68  89.221566  107.0 2019-06-24 10:08:00   
Юрфак       705    0.87  85.471390  107.0 2019-06-24 10:12:00   

                               
                          max  
Факультет                      
Биофак    2019-09-07 14:13:00  
ГУКЦ      2019-09-03 13:40:00  
...                       ...  
Экономфак 2019-10-08 14:51:00  
Юрфак     2019-10-08 15:28:00  

[15 rows x 6 columns]

Группировку можно выполнять сразу по нескольким столбцам:

In [ ]:
groups = df[['ЛД', 'Рейтинг', 'Факультет','Форма обучения', 'Дата создания']].groupby(['Факультет', 'Форма обучения'])
groups.size()

Факультет  Форма обучения
Биофак     заочная            54
           очная             189
                            ... 
Юрфак      заочная           207
           очная             498
Length: 30, dtype: int64

In [ ]:
groups.aggregate(d)

ЛД Рейтинг                         Дата создания  \
                         count     min       mean    max                 min   
Факультет Форма обучения                                                       
Биофак    заочная           54   15.00  74.752593   94.6 2019-06-24 14:08:00   
          очная            189   62.48  88.997672  107.0 2019-06-24 10:14:00   
...                        ...     ...        ...    ...                 ...   
Юрфак     заочная          207    0.87  74.347874  102.9 2019-06-24 11:32:00   
          очная            498   13.88  90.095020  107.0 2019-06-24 10:12:00   

                                              
                                         max  
Факультет Форма обучения                      
Биофак    заочная        2019-09-07 14:13:00  
          очная          2019-09-04 12:47:00  
...                                      ...  
Юрфак     заочная        2019-10-08 15:28:00  
          очная          2019-09-07 13:40:00  

[30 rows x 6 columns]

In [ ]:
df = data[data['Комиссия'] == "Бакалавры/Специалисты"][['ЛД', 'Рейтинг','Факультет', 'Форма обучения','Дата', 'Время']]
df_ = df.groupby(['Дата', 'Факультет', 'Форма обучения']).agg({'ЛД':'count', 'Рейтинг':['min', 'mean', 'max']})
df_.head(30)

ЛД Рейтинг                   
                                      count     min       mean     max
Дата       Факультет   Форма обучения                                 
01.07.2019 Биофак      заочная            3   15.36  61.286667   89.94
                       очная             21   79.96  91.264762  100.90
           ИФКС        заочная           18   11.64  68.211111   89.78
                       очная             36   40.72  80.340556   96.20
           Истфак      заочная            6   78.12  89.466667   98.84
                       очная             39   44.72  91.055385  104.20
           Пединститут заочная           52   15.28  72.610769   93.70
                       очная             90   67.42  89.518333  103.90
                       очно-заочная       1   71.60  71.600000   71.60
           УНИЭК       очная              5   78.78  88.768000  100.20
           Учфин       заочная            1   89.72  89.720000   89.72
                       очная             47   82.24  93.248936  106.50
           ФДиПО       очная              2   15.16  44.660000   74.16
           ФИЯ         очная             67   27.00  93.554030  107.00
           ФМИТ        заочная            4   15.36  49.020000   93.36
                       очная             51   71.98  90.658039  107.00
           Физтех      очная             31   77.60  87.645806   98.70
           Филфак      заочная           28   74.00  84.165000   93.24
                       очная            114   44.84  86.896842  107.00
           Химфак      очная             11   85.04  94.996364  104.90
           Экономфак   заочная            8   20.00  76.772500   95.08
                       очная            114   20.00  91.447544  107.00
           Юрфак       заочная            1   79.28  79.280000   79.28
                       очная             45   47.58  90.695111  106.00
02.07.2019 Биофак      очная             27   79.56  89.171111   97.44
           ИФКС        заочная            8   73.04  77.100000   87.16
                       очная             22   16.96  74.949091   93.22
           Истфак      заочная            4   70.98  79.620000   86.62
                       очная             38   73.46  89.271579  107.00
           Пединститут заочная            5   69.20  74.872000   88.58

Иногда группировка создаёт дополнительные проблемы с обращениям к индексам и, как следствие, к данным, посколку индекс становится многоуровневым объектом `MultiIndex`.

In [ ]:
df_.index

MultiIndex([('01.07.2019',      'Биофак',      'заочная'),
            ('01.07.2019',      'Биофак',        'очная'),
            ('01.07.2019',        'ИФКС',      'заочная'),
            ('01.07.2019',        'ИФКС',        'очная'),
            ('01.07.2019',      'Истфак',      'заочная'),
            ('01.07.2019',      'Истфак',        'очная'),
            ('01.07.2019', 'Пединститут',      'заочная'),
            ('01.07.2019', 'Пединститут',        'очная'),
            ('01.07.2019', 'Пединститут', 'очно-заочная'),
            ('01.07.2019',       'УНИЭК',        'очная'),
            ...
            ('29.06.2019',      'Химфак',        'очная'),
            ('29.06.2019',   'Экономфак',        'очная'),
            ('29.06.2019',       'Юрфак',      'заочная'),
            ('29.06.2019',       'Юрфак',        'очная'),
            ('29.07.2019',      'Биофак',        'очная'),
            ('29.07.2019',       'УНИЭК',        'очная'),
            ('29.07.2019',         'ФИЯ'

 В этом случае можно изменить представление с помощью метода `.unstack()`:

In [ ]:
df_.unstack()

ЛД                              Рейтинг  ...  \
                         count                                  min  ...   
Форма обучения         заочная очная очно-заочная экстернат заочная  ...   
Дата       Факультет                                                 ...   
01.07.2019 Биофак          3.0  21.0          NaN       NaN   15.36  ...   
           ИФКС           18.0  36.0          NaN       NaN   11.64  ...   
           Истфак          6.0  39.0          NaN       NaN   78.12  ...   
           Пединститут    52.0  90.0          1.0       NaN   15.28  ...   
           УНИЭК           NaN   5.0          NaN       NaN     NaN  ...   
...                        ...   ...          ...       ...     ...  ...   
29.07.2019 УНИЭК           NaN  14.0          NaN       NaN     NaN  ...   
           ФИЯ             NaN   1.0          NaN       NaN     NaN  ...   
           Физтех          NaN   8.0          NaN       NaN     NaN  ...   
           Филфак          NaN  22.0          NaN       NaN     NaN  ...   
           Экономфак       NaN   2.0          NaN       NaN     NaN  ...   

                                                                         
                            mean     max                                 
Форма обучения         экстернат заочная   очная очно-заочная экстернат  
Дата       Факультет                                                     
01.07.2019 Биофак            NaN   89.94  100.90          NaN       NaN  
           ИФКС              NaN   89.78   96.20          NaN       NaN  
           Истфак            NaN   98.84  104.20          NaN       NaN  
           Пединститут       NaN   93.70  103.90         71.6       NaN  
           УНИЭК             NaN     NaN  100.20          NaN       NaN  
...                          ...     ...     ...          ...       ...  
29.07.2019 УНИЭК             NaN     NaN   91.58          NaN       NaN  
           ФИЯ               NaN     NaN   90.44          NaN       NaN  
           Физтех            NaN     NaN   90.70          NaN       NaN  
           Филфак            NaN     NaN   93.86          NaN       NaN  
           Экономфак         NaN     NaN   88.80          NaN       NaN  

[377 rows x 16 columns]

In [ ]:
df_.unstack(level=0)

ЛД                                   \
                                count                                    
Дата                       01.07.2019 02.07.2019 02.09.2019 03.07.2019   
Факультет   Форма обучения                                               
Биофак      заочная               3.0        NaN        2.0        3.0   
            очная                21.0       27.0        2.0       19.0   
ГУКЦ        заочная               NaN        NaN        NaN        NaN   
ИФКС        заочная              18.0        8.0       11.0        3.0   
            очная                36.0       22.0        9.0        7.0   
Истфак      заочная               6.0        4.0        1.0        2.0   
            очная                39.0       38.0        1.0       29.0   
Пединститут заочная              52.0        5.0       15.0        9.0   
            очная                90.0       43.0        5.0       25.0   
            очно-заочная          1.0        1.0        NaN        1.0   
УНИЭК       заочная               NaN        NaN        NaN        3.0   
            очная                 5.0        7.0        NaN        9.0   
Учфин       заочная               1.0        1.0        5.0        2.0   
            очная                47.0       58.0        4.0       47.0   
ФДиПО       заочная               NaN        NaN        1.0        NaN   
            очная                 2.0        4.0        1.0        3.0   
ФИЯ         очная                67.0       58.0       17.0       53.0   
ФМИТ        заочная               4.0        NaN        2.0        2.0   
            очная                51.0       32.0        6.0       37.0   
Физтех      заочная               NaN        NaN        2.0        1.0   
            очная                31.0       34.0        2.0       20.0   
            экстернат             NaN        NaN        NaN        NaN   
Филфак      заочная              28.0       16.0        8.0       23.0   
            очная               114.0       85.0        9.0       74.0   
Химфак      заочная               NaN        NaN        3.0        NaN   
            очная                11.0        4.0        NaN        8.0   
Экономфак   заочная               8.0        7.0       15.0        3.0   
            очная               114.0      120.0        4.0       68.0   
Юрфак       заочная               1.0        6.0       17.0        9.0   
            очная                45.0       54.0        9.0       35.0   

                                       ...    Рейтинг                        \
                                       ...        max                         
Дата                       03.09.2019  ... 26.06.2019 27.06.2019 28.06.2019   
Факультет   Форма обучения             ...                                    
Биофак      заочная               NaN  ...      67.64      77.84        NaN   
            очная                 NaN  ...     107.00     100.00      104.0   
ГУКЦ        заочная               3.0  ...        NaN      75.10        NaN   
ИФКС        заочная              15.0  ...        NaN        NaN        NaN   
            очная                 4.0  ...        NaN        NaN        NaN   
Истфак      заочная               1.0  ...      93.56      91.28      102.3   
            очная                 NaN  ...     105.00     104.60      104.5   
Пединститут заочная               9.0  ...        NaN        NaN        NaN   
            очная                 1.0  ...        NaN        NaN        NaN   
            очно-заочная          NaN  ...        NaN        NaN        NaN   
УНИЭК       заочная               NaN  ...      71.12        NaN        NaN   
            очная                 NaN  ...     100.20      76.20      107.0   
Учфин       заочная               4.0  ...      85.74        NaN        NaN   
            очная                 4.0  ...     105.80     107.00      106.7   
ФДиПО       заочная               NaN  ...        NaN        NaN        NaN   
            очная

In [ ]:
df_.unstack(level=1)

ЛД                                ... Рейтинг  \
                           count                                ...     max   
Факультет                 Биофак ГУКЦ  ИФКС Истфак Пединститут  ...  Физтех   
Дата       Форма обучения                                       ...           
01.07.2019 заочная           3.0  NaN  18.0    6.0        52.0  ...     NaN   
           очная            21.0  NaN  36.0   39.0        90.0  ...    98.7   
           очно-заочная      NaN  NaN   NaN    NaN         1.0  ...     NaN   
02.07.2019 заочная           NaN  NaN   8.0    4.0         5.0  ...     NaN   
           очная            27.0  NaN  22.0   38.0        43.0  ...   105.0   
...                          ...  ...   ...    ...         ...  ...     ...   
28.06.2019 заочная           NaN  NaN   NaN    9.0         NaN  ...     NaN   
           очная            13.0  NaN   NaN   29.0         NaN  ...   100.2   
29.06.2019 заочная           NaN  NaN   NaN    1.0         NaN  ...     NaN   
           очная             4.0  NaN   NaN   13.0         NaN  ...   104.0   
29.07.2019 очная            13.0  NaN   NaN    NaN         NaN  ...    90.7   

                                                             
                                                             
Факультет                  Филфак  Химфак Экономфак   Юрфак  
Дата       Форма обучения                                    
01.07.2019 заочная          93.24     NaN     95.08   79.28  
           очная           107.00  104.90    107.00  106.00  
           очно-заочная       NaN     NaN       NaN     NaN  
02.07.2019 заочная         106.50     NaN    106.50   94.74  
           очная           106.50   94.16    106.50  104.90  
...                           ...     ...       ...     ...  
28.06.2019 заочная          99.50     NaN    103.30  102.90  
           очная           105.20  101.00    107.00  107.00  
29.06.2019 заочная          92.22     NaN       NaN   96.50  
           очная            99.00   96.52    107.00  104.00  
29.07.2019 очная            93.86     NaN     88.80     NaN  

[72 rows x 60 columns]

Другой способ, который позволяет получить значеиния индексов определенного уровня из многоуровневого индекса - это применение метода `.get_level_values()` к объекту индекс:

In [ ]:
df_.index.get_level_values(1)

Index(['Биофак', 'Биофак', 'ИФКС', 'ИФКС', 'Истфак', 'Истфак', 'Пединститут',
       'Пединститут', 'Пединститут', 'УНИЭК',
       ...
       'Химфак', 'Экономфак', 'Юрфак', 'Юрфак', 'Биофак', 'УНИЭК', 'ФИЯ',
       'Физтех', 'Филфак', 'Экономфак'],
      dtype='object', name='Факультет', length=595)

Извлечение значений из индексов может пригодиться, например, при построении графиков и использовании их в качестве подписей значений.

## 4. Сортировки

Модуль `Pandas` и его объекты поддерживают сортировки, в том числе многоуровневые:

In [ ]:
df

,ЛД,Рейтинг,Факультет,Форма обучения,Дата,Время
2353,10121БС,96.98,ФИЯ,очная,24.06.2019,9:31
2354,10121БС,96.98,ФИЯ,очная,24.06.2019,9:34
2355,10122БС,94.32,Филфак,очная,24.06.2019,9:39
2356,10122БС,94.32,Филфак,очная,24.06.2019,9:32
2358,10123БС,97.95,Филфак,очная,24.06.2019,10:06
...,...,...,...,...,...,...
12346,14049БС,66.98,Экономфак,заочная,08.10.2019,14:51
12347,14050БС,66.88,Юрфак,заочная,08.10.2019,14:51
12348,14051БС,70.28,Пединститут,заочная,08.10.2019,14:56
12349,14052БС,96.02,ФМИТ,заочная,08.10.2019,15:09


In [ ]:
df.sort_values(by=['Факультет', 'Форма обучения', 'Дата', 'Время'], ascending=False, ignore_index=False)

,ЛД,Рейтинг,Факультет,Форма обучения,Дата,Время
4009,10974БС,99.80,Юрфак,очная,29.06.2019,9:57
4008,10973БС,93.44,Юрфак,очная,29.06.2019,9:54
3990,10964БС,95.28,Юрфак,очная,29.06.2019,9:32
4156,11051БС,98.44,Юрфак,очная,29.06.2019,13:22
4133,11041БС,94.60,Юрфак,очная,29.06.2019,12:22
...,...,...,...,...,...,...
11955,13782БС(ВВ),61.24,Биофак,заочная,02.09.2019,13:22
11464,13578БС,85.26,Биофак,заочная,02.09.2019,12:20
4212,11075БС,89.94,Биофак,заочная,01.07.2019,9:58
4476,11183БС,78.56,Биофак,заочная,01.07.2019,12:38
